In [ ]:
import ray
import numpy as np
from collections import defaultdict



ray.init(log_to_driver=False)



In [ ]:

total_bug_num = 0
success_num = 0

error_num = 0
totoal_identify_num = 0

In [ ]:
@ray.remote(max_calls=6)
def scan_remote(max_step, path_per_node, bug_num, min_qubit_num, max_qubit_num):
    return scan(max_step, path_per_node, bug_num, min_qubit_num, max_qubit_num)

def scan(max_step, path_per_node, bug_num, min_qubit_num, max_qubit_num):
    dataset = get_dataset_bug_detection(min_qubit_num, max_qubit_num)
    # dataset = dataset * 3
    id2circuit_info = {
        circuit_info['id']: circuit_info
        for circuit_info in dataset
    }

    # algs = list(set(id2circuit_info.keys()))

    model = RandomwalkModel(max_step, path_per_node)  #max-step=2 会有14000维，max-step 也有10000维，减少生成的特征的数量``
    # model.batch_train(dataset, 3)
    model.train(dataset)
    model.load_vecs()

    positive_vecs = np.array(model.all_vecs, dtype=np.int64)
    all_qubit_nums = np.array([circuit_info['num_qubits'] for _,_,circuit_info in model.all_instructions], dtype=np.int64)
    # neigative_dataset = copy.deepcopy(dataset)

    num_qubits2positive_vecs = defaultdict(list)
    num_qubits2instructions = defaultdict(list)

    for elm_index, (instruction_index, instruction, circuit_info) in enumerate(model.all_instructions):
        num_qubits2positive_vecs[circuit_info['num_qubits']].append(positive_vecs[elm_index])
        num_qubits2instructions[circuit_info['num_qubits']].append((instruction_index, instruction, circuit_info))

    for gate_num in num_qubits2positive_vecs:
        num_qubits2positive_vecs[gate_num] = jnp.array(num_qubits2positive_vecs[gate_num], dtype=jnp.int64)

    # 构造一个训练集
    # X, Y = [], []
    # for circuit_info in dataset:
    #     gate_vecs = circuit_info['sparse_vecs']
    #     X += gate_vecs
        
    THREADSHOLD = None

    def func_dist(vec1, vec2):
        return jnp.sqrt(sp_dist(vec1, vec2) / (RandomwalkModel.reduced_scaling**2))
        # return sp_cos_dist(vec1, vec2)

    global total_bug_num, success_num, error_num, totoal_identify_num
    
    total_bug_num = 0
    success_num = 0

    error_num = 0
    totoal_identify_num = 0
    
    alg2n_qubit2success_num = defaultdict(lambda : defaultdict(int))
    alg2n_qubit2bug_num = defaultdict(lambda : defaultdict(int))
    alg2n_qubit2error_num = defaultdict(lambda : defaultdict(int))
    alg2n_qubit2identify_num = defaultdict(lambda : defaultdict(int))

    def find_bug(circuit_info, bug_instructions):  # bug_instructions是正样本
        global total_bug_num, success_num, error_num, totoal_identify_num
        alg_name = circuit_info['alg_id']
        circuit_info = model.vectorize(circuit_info)
        gate_vecs = circuit_info['sparse_vecs'] #.reshape(-1, 100)
        bug_circuit = circuit_info['qiskit_circuit']
        num_qubits = circuit_info['num_qubits']
        
        # print(bug_circuit)
        # assert len(bug_instructions) == 1

        instruction2nearest_circuits = []

        print('start', circuit_info['id'])
        
        for analyzed_vec_index, analyzed_vec in enumerate(gate_vecs):
            this_instruction = circuit_info['instructions'][analyzed_vec_index]
            has_bug = analyzed_vec_index in bug_instructions
            
            # if has_bug:
            #     print('-----------next has bug-------')
            
            dists = vmap(func_dist, in_axes=(0, None), out_axes=0)(num_qubits2positive_vecs[num_qubits], analyzed_vec)
            # dists = vmap(func_dist, in_axes=(0, None), out_axes=0)(positive_vecs, analyzed_vec)
            # dists += (1- (all_qubit_nums == circuit_info['num_qubits'])) * 20 # 先mask个提高下准确度
            
            dist_indexs = np.argsort(dists)[:3]  # 小的在前面
            nearest_dists = dists[dist_indexs]
            
            dist_indexs = dist_indexs[nearest_dists < 2]
            nearest_dists = dists[dist_indexs]
            
            nearest_positive_instructions = [
                num_qubits2instructions[num_qubits][_index]
                for _index in dist_indexs
            ]
            nearest_circuits = [
                elm[2]['id']
                for elm in nearest_positive_instructions
            ]
            
            nearest_circuit_instruction_index = [
                elm[0]
                for elm in nearest_positive_instructions
            ]
            
            # print(analyzed_vec_index, this_instruction['name'], this_instruction['qubits'], nearest_dists, nearest_circuits, nearest_circuit_instruction_index)

            if np.abs(analyzed_vec_index - bug_instructions[0]) <= 2:

                if analyzed_vec_index in bug_instructions:
                    print('next is bug') 
                
                print(circuit_info['id'], nearest_dists)
                print('this', circuit_info['instructions'][analyzed_vec_index])
                # if analyzed_vec_index in bug_instructions:
                #     print(circuit_info['instructions'][analyzed_vec_index]['original'])
                

                print('nearest_circuits', nearest_circuits) # 相近的电路
                print('nearest_positive_instructions', nearest_circuit_instruction_index)
                # print(nearest_positive_instructions[0]['path_indexs'][nearest_positive_instructions[0]['id']], circuit_info['path_indexs'][analyzed_vec_index])

                print('\n')

            nearest_circuits = set(nearest_circuits)
            
            instruction2nearest_circuits.append(nearest_circuits)
            
        # print('finish\n\n')
        #     nearest_index = np.argmin(dists) 
        #     nearest_dist = dists[nearest_index]
        #     # nearest_positive_instruction = model.all_instructions[nearest_index]
        #     nearest_positive_instruction = num_qubits2instructions[num_qubits][nearest_index]
        #     nearest_circuit = nearest_positive_instruction[2]
        #     nearest_circuits.append(nearest_circuit['id'])

        # global total_bug_num, success_num, error_num, totoal_identify_num

        for index, nearest_circuits in enumerate(instruction2nearest_circuits):

            neighbor_nearest_circuits = []
            for nearest_circuit_set in instruction2nearest_circuits[index-6:index] + instruction2nearest_circuits[index+1: index+6]: 
                neighbor_nearest_circuits += list(nearest_circuit_set)
            
            neighbor_mode_nearest_circuit1 = statistics.mode(neighbor_nearest_circuits)
            neighbor_nearest_circuits = [elm for elm in neighbor_nearest_circuits if elm != neighbor_mode_nearest_circuit1]
            
            if len(neighbor_nearest_circuits) > 0:
                neighbor_mode_nearest_circuit2 = statistics.mode(neighbor_nearest_circuits)
            else:
                neighbor_mode_nearest_circuit2 = None
            # print(neighbor_mode_nearest_circuit, nearest_circuit, circuit_info['id'], index in bug_instructions)

            if neighbor_mode_nearest_circuit1 not in nearest_circuits:
                if neighbor_mode_nearest_circuit2 is None:
                    if index not in bug_instructions:
                        error_num += 1
                        alg2n_qubit2error_num[alg_name][num_qubits] += 1
                elif neighbor_mode_nearest_circuit2 not in nearest_circuits:
                    if index not in bug_instructions:
                        error_num += 1
                        alg2n_qubit2error_num[alg_name][num_qubits] += 1
                # else:
                #     success_num += 1
                    
            totoal_identify_num += 1
            alg2n_qubit2identify_num[alg_name][num_qubits] += 1
        # # nearest_circuits, 
        # print(circuit_info['id'], [nearest_circuits[index] for index in bug_instructions], '\n')
        
        
        total_bug_num += 0 if len(bug_instructions) == 0 else 1#len(bug_instructions)
        alg2n_qubit2bug_num[alg_name][num_qubits] += 0 if len(bug_instructions) == 0 else 1
        for index in bug_instructions:
            if circuit_info['original_id'] not in instruction2nearest_circuits[index]:
                success_num += 1
                alg2n_qubit2success_num[alg_name][num_qubits] += 1
                break

        print(bug_num, min_qubit_num, max_qubit_num, 'success rate:', success_num / total_bug_num)
        print(bug_num, min_qubit_num, max_qubit_num, 'error rate:', error_num / totoal_identify_num)
        return bug_circuit
        # return potential_bugs


    from dataset.dataset1 import hamiltonian_simulation, ising, swap, QAOA_maxcut, qknn, qsvm, vqc
    from dataset.dataset2 import qnn, qugan, simon
    from dataset.get_data import get_bitstr

    def construct_negative(circuit_info, bug_num):
        '''bug的宽度'''
        
        circuit = QuantumCircuit(circuit_info['num_qubits'])

        start_instruction = random.randint(0, len(circuit_info['instructions'])-1-bug_num)
        end_instruction = start_instruction + bug_num
        bug_instructions = list(range(start_instruction, end_instruction))
        # bug_instructions = list(range(len(circuit_info['instructions'])))
        # random.shuffle(bug_instructions)
        # bug_instructions = bug_instructions[:bug_num]

        for layer, layer_instructions in enumerate(circuit_info['layer2instructions']):
            for instruction in layer_instructions:
                name = instruction['name']
                qubits = instruction['qubits']
                params = instruction['params']
                id = instruction['id']

                add_bug = False
                if id in bug_instructions: #and name in ('rx', 'ry', 'rz', 'h', 'cz', 'cx'):
                    # if name in ('rx', 'ry', 'rz', 'h'):
                    #     name = random.choice(['rx', 'ry', 'rz','h'])
                    # if name in ('cz', 'cx'):
                    #     name = random.choice(['cz', 'cx'])
                    name = random.choice(['rx', 'ry', 'rz', 'h', 'cz', 'cx'])
                    add_bug = True

                    params = [random.random()]
                    qubit1 = random.randint(0, circuit_info['num_qubits']-1)
                    qubit2 = random.choice([qubit for qubit in range(circuit_info['num_qubits']) if qubit != qubit1])
                    qubits = [qubit1, qubit2]

                    # bug_num -= 1

                if name in ('rx', 'ry', 'rz'):
                    circuit.__getattribute__(name)(params[0], qubits[0])
                elif name in ('cz', 'cx'):
                    circuit.__getattribute__(name)(qubits[0], qubits[1])
                elif name in ('h'):
                    circuit.__getattribute__(name)(qubits[0])
                else:
                    circuit.__getattribute__(name)(*(params + qubits))


                circuit[len(circuit)-1].operation.label = f'bug-{name}' if add_bug else None
            circuit.barrier()
        
        bug_circuit_info = parse_circuit(circuit)
        for bug_instruction in bug_instructions:
            bug_circuit_info['instructions'][bug_instruction]['original'] = circuit_info['instructions'][bug_instruction]
        bug_circuit_info['id'] = circuit_info['id'] + f'_bug_{bug_instructions}'
        bug_circuit_info['original_id'] = circuit_info['id']

        return bug_circuit_info, bug_instructions


    def bug_detect(al_name, n_qubit):
        circuit_id = f'{al_name}_{n_qubit}'

        positive_circuit_info = id2circuit_info[circuit_id]
        # 这里就只管随机替换吧
        bug_circuit_info, bug_instructions = construct_negative(positive_circuit_info, bug_num = bug_num) 

        bug_circuit_info['alg_id'] = al_name
        # compare_circuits(positive_circuit_info['qiskit_circuit'], bug_circuit_info['qiskit_circuit'])

        identified_circuit_info = find_bug(bug_circuit_info, bug_instructions)

        # test_info = list(range(len(bug_circuit['qiskit_circuit'])))
        # plot_circuit_info(bug_circuit, test_info)

        # fig = bug_circuit['qiskit_circuit'].draw(output='mpl')
        # fig.savefig(f'./result/{al_name}_{n_qubit}_type{bug_type}.png')
        # bug_circuit = find_bug(bug_circuit)
        # fig = bug_circuit.draw(output='mpl', style={"displaycolor":{"bug":"#FF0000"}})
        # fig.savefig(f'./result/{al_name}_{n_qubit}_type{bug_type}_bug.png')


    # bug_detect('hamiltonian_simulation', 4)

    # al_names = ['hamiltonian_simulation', 'qsvm']
    al_names = ['qft']
    for al_name in al_names:
        for n_qubit in range(min_qubit_num, max_qubit_num):
                bug_detect(al_name, n_qubit)


    print('bug_size', bug_num)
    print('alg2n_qubit2success_num=', alg2n_qubit2success_num)
    print('alg2n_qubit2bug_num=', alg2n_qubit2bug_num)
    print('alg2n_qubit2error_num=', alg2n_qubit2error_num)
    print('alg2n_qubit2identify_num=', alg2n_qubit2identify_num)
    
    return alg2n_qubit2success_num, alg2n_qubit2bug_num, alg2n_qubit2error_num, alg2n_qubit2identify_num


In [ ]:


results = {}
step_num = 3
for min_qubit_num in range(10, 50, step_num):
    max_qubit_num = min_qubit_num + step_num
    for max_step in range(3, 5):
        path_per_node = max_step * 16
        for bug_num in range(1, 4):
            key = (min_qubit_num, max_qubit_num, bug_num, max_step)
            if os.path.exists(f'./temp/debug_result_{key}.pkl'):
                continue
            future = scan_remote.remote(max_step, path_per_node, bug_num, min_qubit_num, max_qubit_num)
            # # scan(bug_num, min_qubit_num, max_qubit_num)
            results[(min_qubit_num, max_qubit_num, bug_num, max_step)] = future
            # pass
# future = scan_remote.remote(max_step, path_per_node, bug_num, min_qubit_num, max_qubit_num)
# future = scan(3, 50, 2, 10, 15)
# results[(min_qubit_num, max_qubit_num, bug_num, max_step)] = future


def to_dict(obj):
    new_obj = {}
    for key, elm in obj.items():
        if isinstance(elm, (dict, defaultdict)):
            new_obj[key] = to_dict(elm)
        else:
            new_obj[key] = elm
    return new_obj

import pickle
for key, future in results.items():
    # alg2n_qubit2success_num, alg2n_qubit2bug_num, alg2n_qubit2error_num, alg2n_qubit2identify_num = future.result()
    # result = results[key].get()
    result = results[key]
    print(result)
    alg2n_qubit2success_num, alg2n_qubit2bug_num, alg2n_qubit2error_num, alg2n_qubit2identify_num = result
    # ray.get(results[key])
    
    result  = {
        'key': key,
        'alg2n_qubit2success_num': to_dict(alg2n_qubit2success_num),
        'alg2n_qubit2bug_num': to_dict(alg2n_qubit2bug_num), 
        'alg2n_qubit2error_num': to_dict(alg2n_qubit2error_num), 
        'alg2n_qubit2identify_num': to_dict(alg2n_qubit2identify_num) 
    }
    results[key] = result
    
#     with open(f'./temp/debug_result_{key}.pkl', 'wb') as file:
#         pickle.dump(result, file)

# with open('debug_result.pkl', 'wb') as file:
#     pickle.dump(results, file)

In [ ]:
qubit_step = 5
min_qubit, max_qubit  = 10, 37
bug_num_ranges = [
    (start, start + qubit_step)
    for start in range(min_qubit, max_qubit, qubit_step)
]

In [ ]:
def findRange(value, qubit_ranges):
    for start, end in qubit_ranges:
        if value >= start and value < end:
            return (start, end)
    return None

In [ ]:
alg2n_qubit2success_num = defaultdict(lambda: defaultdict(int))
alg2n_qubit2bug_num = defaultdict(lambda: defaultdict(int))

# alg_names = set()
# qubit_nums = set()
for result in all_results:
    min_qubit_num, max_qubit_num, bug_num, max_step = result['key']
    # if max_step != ??:
    #   continue
    for alg_name, n_qubit2success_num in result['alg2n_qubit2success_num'].items():
        for n_qubit, success_num in n_qubit2success_num.items():
            # alg2n_qubit2success_num[alg_name][n_qubit] += success_num 
            alg2n_qubit2success_num[alg_name][findRange(n_qubit,bug_num_ranges)] += success_num 
            
        # alg_names.add(alg_name)
        # qubit_nums.add((min_qubit_num,max_qubit_num))

    for alg_name, n_qubit2bug_num in result['alg2n_qubit2bug_num'].items():
        for n_qubit, bug_num in n_qubit2bug_num.items():
            # alg2n_qubit2bug_num[alg_name][n_qubit] += bug_num
            alg2n_qubit2bug_num[alg_name][findRange(n_qubit,bug_num_ranges)] += bug_num  
            
# alg_names = list(alg_names)
# qubit_nums = list(qubit_nums)
# alg_names.sort()
# qubit_nums.sort()
# alg_names, qubit_nums

In [ ]:
def combine(dict1, dict2, default_value = 1):
    all_values = []
    new_dict = defaultdict(lambda: defaultdict(lambda: default_value))
    for k1, v1 in dict1.items():
        for k2, v2 in v1.items():
            new_dict[k1][k2] = v2 / dict2[k1][k2]
            all_values.append(v2 / dict2[k1][k2])
    
    return new_dict, np.array(all_values)

In [ ]:
alg2n_qubit2success_rate, all_values = combine(alg2n_qubit2success_num, alg2n_qubit2bug_num)
alg2n_qubit2success_rate, np.mean(all_values)

In [ ]:
# index: qubit_nums:
# cols: alg_names

alg_names = list(alg2n_qubit2success_rate.keys())
# qubit_nums = list(range(10, 30))
# qubit_nums.sort()
alg_names.sort()

# alg_names, qubit_nums

data = np.zeros((len(bug_num_ranges) , len(alg_names))) # 先index, 后column

for r_i, qubit_num in enumerate(bug_num_ranges):
    for c_i, alg_name in enumerate(alg_names):
        data[r_i][c_i] = alg2n_qubit2success_rate[alg_name][qubit_num]

df = pd.DataFrame(data, index=bug_num_ranges, columns=alg_names)
fig = sns.heatmap(data=df,cmap='OrRd')
fig.get_figure().savefig('success_rate.svg')


In [ ]:
alg2n_qubit2error_num = defaultdict(lambda: defaultdict(int))
alg2n_qubit2identify_num = defaultdict(lambda: defaultdict(int))

alg_names = set()
qubit_nums = set()
for result in all_results:
    min_qubit_num, max_qubit_num, identify_num, max_step = result['key']
    # if max_step != ??:
    #   continue
    for alg_name, n_qubit2error_num in result['alg2n_qubit2error_num'].items():
        for n_qubit, error_num in n_qubit2error_num.items():
            # alg2n_qubit2success_num[alg_name][n_qubit] += success_num 
            alg2n_qubit2error_num[alg_name][findRange(n_qubit,bug_num_ranges)] += error_num 
            
        # alg_names.add(alg_name)
        # qubit_nums.add((min_qubit_num,max_qubit_num))

    for alg_name, n_qubit2identify_num in result['alg2n_qubit2identify_num'].items():
        for n_qubit, identify_num in n_qubit2identify_num.items():
            # alg2n_qubit2bug_num[alg_name][n_qubit] += bug_num
            alg2n_qubit2identify_num[alg_name][findRange(n_qubit,bug_num_ranges)] += identify_num  
            
# alg_names = list(alg_names)
# qubit_nums = list(qubit_nums)
# alg_names.sort()
# qubit_nums.sort()
# alg_names, qubit_nums

In [ ]:
alg2n_qubit2error_rate, all_values = combine(alg2n_qubit2error_num, alg2n_qubit2identify_num, 0)
alg2n_qubit2error_rate, np.mean(all_values)

In [ ]:
# index: qubit_nums:
# cols: alg_names

alg_names = list(alg2n_qubit2error_rate.keys())
# qubit_nums = list(range(10, 30))
# qubit_nums.sort()
alg_names.sort()

# alg_names, qubit_nums

data = np.zeros((len(bug_num_ranges) , len(alg_names))) # 先index, 后column 

for r_i, qubit_num in enumerate(bug_num_ranges):
    for c_i, alg_name in enumerate(alg_names):
        data[r_i][c_i] = alg2n_qubit2error_rate[alg_name][qubit_num]

# cbar
df = pd.DataFrame(data, index=bug_num_ranges, columns=alg_names)
fig = sns.heatmap(data=df,cmap='OrRd')
fig.get_figure().savefig('error_rate.svg')

In [ ]:
alg2n_qubit2success_num = defaultdict(lambda: defaultdict(int))
alg2n_qubit2bug_num = defaultdict(lambda: defaultdict(int))

for result in all_results:
    min_qubit_num, max_qubit_num, bug_num_index, max_step = result['key']
    # if max_step != ??:
    #   continue
    for alg_name, n_qubit2success_num in result['alg2n_qubit2success_num'].items():
        for n_qubit, success_num in n_qubit2success_num.items():
            # alg2n_qubit2success_num[alg_name][n_qubit] += success_num 
            alg2n_qubit2success_num[alg_name][bug_num_index] += success_num 
            
  
    for alg_name, n_qubit2bug_num in result['alg2n_qubit2bug_num'].items():
        for n_qubit, bug_num in n_qubit2bug_num.items():
            # alg2n_qubit2bug_num[alg_name][n_qubit] += bug_num
            alg2n_qubit2bug_num[alg_name][bug_num_index] += bug_num  
            
alg2n_qubit2success_rate, all_values = combine(alg2n_qubit2success_num, alg2n_qubit2bug_num)
alg2n_qubit2success_rate, np.mean(all_values)

In [ ]:
# index: qubit_nums:
# cols: alg_names

alg_names = list(alg2n_qubit2success_rate.keys())
# qubit_nums = list(range(10, 30))
# qubit_nums.sort()
alg_names.sort()

# alg_names, qubit_nums
bug_num_ranges = [1,2,3]
data = np.zeros((len(bug_num_ranges) , len(alg_names))) # 先index, 后column

for r_i, qubit_num in enumerate(bug_num_ranges):
    for c_i, alg_name in enumerate(alg_names):
        data[r_i][c_i] = alg2n_qubit2success_rate[alg_name][qubit_num]

df = pd.DataFrame(data, index=bug_num_ranges, columns=alg_names)
fig = sns.heatmap(data=df,cmap='OrRd')
fig.get_figure().savefig('success_rate.svg')
